In [2]:
from mylosses import SupConLoss as suploss
from resnet_big import SupConResNet 

In [3]:
import torch
from torchvision import transforms, datasets
from torch import randperm
from util import TwoCropTransform
from my_supcon import CustomSubset

def set_loader():
    # construct data loader
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)

    normalize = transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])

    train_transform = transforms.Compose([
        transforms.Resize([32,32]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])

    img_dataset = datasets.ImageFolder(r'D:\research\su\data\Knee\Digital_Knee_X-ray_Images\MedicalExpert-I')
    class_names = img_dataset.classes
    size = len(img_dataset)
    img_dataset = torch.utils.data.Subset(img_dataset, randperm(size).tolist())
    train_indices = range(0, int(size*0.5))
    train_set = CustomSubset(img_dataset, train_indices, TwoCropTransform(train_transform))
    print(f'train dataset size : {len(train_set)}')
    print(f'classes names : {class_names}')
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=10,
                                                shuffle=True, pin_memory=True,num_workers=0)
    return train_loader

In [7]:
loader = set_loader()

train dataset size : 825
classes names : ['0Normal', '1Doubtful', '2Mild', '3Moderate', '4Severe']


In [10]:
model = SupConResNet(name='resnet18')

In [9]:
def test(loader, model, loss):
    for images, labels in loader:
        print('image list size:', len(images))
        images = torch.cat([images[0], images[1]], dim=0)
        print('input images size:', images.shape)
        bsz = labels.shape[0]
        print('bsz:',bsz)
        features = model(images)
        print('output features shape:', features.shape)
        f1, f2 = torch.split(features, [bsz, bsz], dim=0)
        print(f'f1 shape:{f1.shape}, f2 shape:{f2.shape}')
        features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
        print('concatted feature shape:', features.shape)
        score = loss(features, labels)
        print('loss:', score)
        break

In [8]:
import numpy as np

def L2(X,Y):
    square_sum_X = torch.sum(X ** 2, dim=1, keepdim=True)
    square_sum_Y = torch.sum(Y ** 2, dim=0, keepdim=True)
    distances = square_sum_X + square_sum_Y - 2.0 * torch.matmul(X, Y)
    distances = torch.sqrt(torch.clamp(distances, min=0.0))
    return distances

In [11]:
from mylosses import SupConLoss as myloss
loss = myloss(dis_func = L2)
test(loader, model, loss)

image list size: 2
input images size: torch.Size([20, 3, 32, 32])
bsz: 10
output features shape: torch.Size([20, 128])
f1 shape:torch.Size([10, 128]), f2 shape:torch.Size([10, 128])
concatted feature shape: torch.Size([10, 2, 128])
loss: tensor(3.3806, grad_fn=<MeanBackward0>)


In [ ]:
# from main_linear import train,set_model,set_optimizer,set_loader
# train(loader,model,)